<a href="https://colab.research.google.com/github/frankausberlin/notebook-collection/blob/main/snippetpearls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table width=1000><tr></tr><tr><td align='center'><font size='+10'><b>
SnippetPearls</b></font></td></tr><tr></tr><tr><td align='center'><p>
This is a jupyter notebook with several and I hope useful<br> code snippets around very interesting technologies.</td></tr><tr></tr><tr><td align='center'>
optimized for google colab</td></tr></table><br><br>

---

ps.

There are problems with the tab-widget from ipywidgets. In some constellations it will not be rendered (local runtime, chrome on windows, maybe more). But it works correctly (it just draw the content one after the other and not in tabs).

you can look [here](https://github.com/googlecolab/colabtools/issues/3105)

<br>
<font size='+2'>Installation<br>

It's realy easy: 
1. Copy the url of your (or this) notebook,
2. go to menu tools / settings / websites
3. and past your url to field 'Custom snippet notebook url'.

It will now, after reloading the page, treat all text cells with chapter entries ('#') and their subsequent code cells as a snippet and display them in the colab snippet selector

# _sp: command shell

In [ ]:
#@markdown command shell { form-width: "300px" }
#@markdown * using [jQuery Terminal](https://terminal.jcubic.pl/) - example on [github](https://github.com/alessandrobessi/colab-shell)
#
#
%matplotlib inline
import                            os
from   google.colab       import  widgets, output
from   IPython.display    import  display, HTML, JSON
from   subprocess         import  getoutput

width  = 900 #@param {type: "slider", min: 400, max: 1200, step: 100}
height = 400 #@param {type: "slider", min: 100, max: 800, step: 100}
printHistory = True #@param {type:"boolean"}

# function for invoke-callback-mechanism
def shell(command):
  # print history an reselect command tab
  if printHistory:
    with tabs.output_to('history'): print(command)
    with tabs.output_to('command'): pass

  # catch cd-command 
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])

  # all other commands
  return JSON([getoutput(command)])

# register to invoke
output.register_callback('shell', shell)

# the tabs
tabs = widgets.TabBar(['command','history'])

# create command tab using javascript with 'display(HTML(...'
with tabs.output_to('command'): display(HTML('''
  <div id=term_demo></div>
  <script src="https://code.jquery.com/jquery-latest.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
  <link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
  <script>
    $('#term_demo').terminal(async function(command) {
        if (command !== '') {
            try {
                let res = await google.colab.kernel.invokeFunction('shell', [command])
                let out = res.data['application/json'][0]
                this.echo(new String(out))
            } catch(e) {
                this.error(new String(e));
            }
        } else {
            this.echo('');
        }
    }, {
        greetings: 'Welcome to Colab Shell',
        name: 'colab_shell',
        height: '''+str(height)+''', 
        width: '''+str(width)+''',
        prompt: 'colab > '
    });
  </script>'''))

# _sp: dictor

In [ ]:
#@markdown dictor 
#@markdown * A little dict editor.
#@markdown * Using ipywidgets.
#@markdown * Easily expandable and customizable.
 
from ipywidgets import Tab, Text, Box, VBox, HBox, Button, Textarea, Layout, Label

content     = {}
buttonAdd   = Button   (description='Add', disabled=True)
buttonDel   = Button   (description='Del', disabled=True)
buttonDoit  = Button   (description='DoIt')
name        = Text     (description='Name', layout=Layout(width='550px'))
note        = Textarea (description='Note', layout=Layout(width='550px',height='100px'))
properties  = VBox     (children=[HBox(children=[buttonAdd,buttonDel,buttonDoit]), name, note])
buttons     = Box      (layout=Layout(display='flex', flex_flow='wrap'))
contentTab  = Tab      (children=[VBox(children=[Textarea()])],layout=Layout(display='flex', flex_flow='wrap') )
dictor      = VBox     ([buttons,properties,contentTab])


def tabClick (b):
  if b.new > 0: 
    buttons.children[b.new-1].click()
  else: 
    name.value = ''
    for w in list(buttons.children): w.button_style = ''
    contentTab.children[0].children[0].value = ''


def buttonClick (b):
  b.button_style = 'info'
  name.value = b.description
  note.value = content[b.description]
  p = -1
  for i,w in enumerate (list (contentTab.children)):
    if i > 0 and w.children[0].value == b.description: 
      contentTab.selected_index = i


def commandClick (b):
  if b.description == 'DoIt':
    contentTab.selected_index = 0
    contentTab.children[0].children[0].value = str(content)

  if b.description == 'Add':
    # new button
    tmpBut = Button (description  = name.value,
                     button_style = 'info',
                     layout       = Layout(display='flex',width='auto') )
    tmpBut.on_click(buttonClick)
    for w in list(buttons.children): w.button_style = ''
    buttons.children    = tuple(list(buttons.children) + [tmpBut])
    buttonAdd.disabled  = True
    buttonDel.disabled  = False
    buttonDoit.disabled = False
    content[name.value] = note.value
    contentTab.children = tuple(list(contentTab.children) + [VBox(children=[Label(value=name.value)])])
    contentTab.selected_index = len (contentTab.children) -1

  if b.description == 'Del':
    # search button by description
    p = -1
    for i in range (len(buttons.children)):
      if buttons.children[i].description == name.value:
        p = i
    # delete from list
    buttons.children    = tuple ([e for i,e in enumerate(buttons.children)    if i != p])
    contentTab.children = tuple ([e for i,e in enumerate(contentTab.children) if i != p+1])
    del content[name.value]
    b.disabled = True
    name.value = ''
    note.value = ''


def nameChange (change):
  # empty name
  if change.new == '':
    buttonDel.disabled=True
    buttonAdd.disabled=True
    note.value = ''
    contentTab.selected_index = 0
    return
  
  # button states
  buttonAdd.disabled  = False
  buttonDel.disabled  = True
  note.value = ''
  for w in list(buttons.children): w.button_style = ''

  # search name in content
  for i in range (len(buttons.children)):
    # and show it if found
    if buttons.children[i].description == change.new:
      note.value          = content[change.new]
      buttonAdd.disabled  = True
      buttonDel.disabled  = False
      buttons.children[i].button_style = 'info'
      break


def noteChange (change):
  if change.new == '' or name.value == '': return
  # every change is written
  content[name.value] = change.new


buttonAdd.on_click(commandClick)
buttonDel.on_click(commandClick)
buttonDoit.on_click(commandClick)
name.observe(nameChange,names=['value'])
note.observe(noteChange,names=['value'])
contentTab.observe (tabClick, names='selected_index')

dictor

# _sp: image search

In [ ]:
#@markdown image search<br> { form-width: "300px" }
#@markdown This little snippet does a picture search with DuckDuckGo (jpg, png, gif, bmp) and show the results (thumbnails). 
#@markdown * It creates a folder content/images (local runtime: ~/labor/content/images) and there a new folder for every search.
#@markdown * The image thumbnails (and optional the images) will be downloaded in the search folder. 
#@markdown * It will generate a bash script (easy remove bad pictures) for downloading the images later (or again).

import                            time, json, re, ipywidgets, os, \
                                  requests, matplotlib
from matplotlib           import  pyplot as plt
from urllib.parse         import  urlencode, quote, urlparse, urlunparse
from urllib.request       import  build_opener, Request
from urllib.error         import  HTTPError, URLError
from google.colab         import  widgets
from PIL                  import  Image
from tqdm                 import  tqdm

query                     = 'angry cat' #@param {type:"string"}
results                   = 12 #@param {type: "slider", min: 1, max: 999, step: 1}
downloadImages            = False #@param {type:"boolean"}
                            #@markdown **thumbnails display**
columns                   = 6 #@param {type: "slider", min: 1, max: 20, step: 1}
rows                      = 3 #@param {type: "slider", min: 1, max: 100, step: 1}

##########################################################################################################
########################## reversed from https://course.fast.ai/ #########################################
##########################################################################################################
url_default_headers = {
    "Accept":
    "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

ExceptionsHTTP = {}

def urlquote(url):
    "Update url's path with `urllib.parse.quote`"
    subdelims = "!$&'()*+,;="
    gendelims = ":?#[]@"
    safe = subdelims+gendelims+"%/"
    p = list(urlparse(url))
    p[2] = quote(p[2], safe=safe)
    for i in range(3,6): p[i] = quote(p[i], safe=safe)
    return urlunparse(p)

def loads(s, **kw):
    "Same as `json.loads`, but handles `None`"
    if not s: return {}
    try: import ujson as json
    except ModuleNotFoundError: import json
    return json.loads(s, **kw)

def urlwrap(url, data=None, headers=None):
    "Wrap `url` in a urllib `Request` with `urlquote`"
    return url if isinstance(url,Request) else Request(urlquote(url), data=data, headers=headers or {})

def urlopener():
    _opener = build_opener()
    _opener.addheaders = list(url_default_headers.items())
    return _opener

def urlopen(url, data=None, headers=None, timeout=None, **kwargs):
    "Like `urllib.request.urlopen`, but first `urlwrap` the `url`, and encode `data`"
    if kwargs and not data: data=kwargs
    if data is not None:
        if not isinstance(data, (str,bytes)): data = urlencode(data)
        if not isinstance(data, bytes): data = data.encode('ascii')
    try: return urlopener().open(urlwrap(url, data=data, headers=headers), timeout=timeout)
    except HTTPError as e: 
        e.msg += f"\n====Error Body====\n{e.read().decode(errors='ignore')}"
        raise

def urlread(url, data=None, headers=None, decode=True, return_json=False, return_headers=False, timeout=None, **kwargs):
    "Retrieve `url`, using `data` dict or `kwargs` to `POST` if present"
    try:
        with urlopen(url, data=data, headers=headers, timeout=timeout, **kwargs) as u: res,hdrs = u.read(),u.headers
    except HTTPError as e:
        if 400 <= e.code < 500: raise ExceptionsHTTP[e.code](e.url, e.hdrs, e.fp) from None
        else: raise

    if decode: res = res.decode()
    if return_json: res = loads(res)
    return (res,dict(hdrs)) if return_headers else res

def isddg(term, max_images=200):
    "Search for `term` with DuckDuckGo and return a unique urls of about `max_images` images"
    assert max_images<1000
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    assert searchObj
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    xu,data = list(),{'next':1} # little change: don't use the mysterious L class
    headers = dict(referer='https://duckduckgo.com/')
    while len(xu)<max_images and 'next' in data:
        try:
            res = urlread(requestUrl, data=params, headers=headers)
            data = json.loads(res) if res else {}
            for i in range(len(data['results'])):
              xu.append(data['results'][i])
            requestUrl = url + data['next']
        except (URLError,HTTPError): pass
        time.sleep(1)
    return xu[:max_images]
##########################################################################################################
##########################################################################################################
##########################################################################################################

# download the ddg results page (links)
links                 = isddg (query,results)
if downloadImages:    print (f'checking {len(links)} links - start download thumbnails and images')
else:                 print (f'checking {len(links)} links - start download thumbnails')

# mkdir
qfolder       = query.replace (' ','')
if os.path.exists('/content'):
  os.makedirs   ('/content/images',exist_ok=True)
  os.makedirs   (f'/content/images/{qfolder}',exist_ok=True)
  os.chdir      (f'/content/images/{qfolder}')
else:
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content',exist_ok=True)
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content/images',exist_ok=True)
  os.makedirs   (f'{os.path.expanduser("~")}/labor/content/images/{qfolder}',exist_ok=True)
  os.chdir      (f'{os.path.expanduser("~")}/labor/content/images/{qfolder}')


# inits
pos, bad, thumbs, bash = 0, 0, [], f'#!/bin/bash\nmkdir -p {qfolder}\n'

# loop the links
for all in tqdm(links): 

  # extract infos
  url  = all['image']
  turl = all['thumbnail']
  ext  = url.split('.')[-1]
  if '?' in ext: ext = ext.split('?')[0]
  dest = f"{qfolder}{pos-bad}.{ext}"
  pos+=1

  # check extension
  if ext.lower() in 'jpg png gif bmp':

    # download image und thumbnail
    try:

      # image
      if downloadImages:
        im = Image.open(requests.get(url, stream = True).raw)
        im.save(dest)

      # bash script entry
      bash += f'# Picture {dest}\ncurl "{url}" --output {qfolder}/{dest}\n' 

      # thumbnail
      im = Image.open(requests.get(turl, stream = True).raw)
      im.save('thumb'+dest) 
      thumbs.append('thumb'+dest)

    # count bad links
    except Exception:
      bad += 1
  else:
    bad += 1

# output infos
if not downloadImages:  txt = f'downloaded {len(thumbs)} thumbnails'
else:                   txt = f'downloaded {len(thumbs)} images and thumbnails'   
print                   (f'\n\n\x1b[32m{txt}\x1b[0m - \x1b[91mfound {bad} bad links!!!\x1b[0m')

# write bash 
if os.path.exists('/content'):
  print    (f'Create bash script: \x1b[34m/content/images/dl_{qfolder}.sh\x1b[0m')
  open     (f'/content/images/dl_{qfolder}.sh', 'w').write(bash)
else:
  print    (f'Create bash script: \x1b[34m~/labor/content/images/dl_{qfolder}.sh\x1b[0m')
  open     (f'{os.path.expanduser("~")}/labor/content/images/dl_{qfolder}.sh', 'w').write(bash)


# build the thumbnails tabs
titles, len_thumbs = [], len(thumbs)
for i in range(0,len_thumbs,(columns*rows)):
  x = ((i//(columns*rows))+1)*(columns*rows)-1
  if x >= len_thumbs: x = len_thumbs-1
  titles.append(f'{i}-{x}')
tabs = widgets.TabBar(titles)

# title font
matplotlib.rc('font', **{'sans-serif':    'DejaVu Sans',
                          'family':        'sans-serif',
                          'weight':        'bold', 
                          'size':          (columns+10 if columns > 12 else 16) })  

# fill the tabs
for i,t in enumerate(titles):
  with tabs.output_to (t): 
    
    # figure dimension (inch)
    fig = plt.figure(figsize=(4*columns, 4.25*rows))
    
    # all images for the tab i
    for j in range(columns*rows):
      x = i*(columns*rows)+j

      # subplot image
      if x < len_thumbs:
        fig.add_subplot(rows, columns, j+1, xticks=[], yticks=[], title=f'{x}')
        im = Image.open (thumbs[x])
        plt.imshow(im)

# cd to image folder         
if os.path.exists('/content'): os.chdir (f'/content/images')
else: os.chdir (f'{os.path.expanduser("~")}/labor/content/images')

#_sp: breakpoint

In [ ]:
#@markdown debug { form-width: "700px" }
#@markdown * When an error happens you can run **%debug** in a new code cell.
#@markdown * Or you set a breakpoint with ```from IPython.core.debugger import Pdb; Pdb().set_trace()```
#@markdown * Some commands:<br><table align='left'><tr><td><b>'q': quit</b> - Exit the debugger.<br>_</td><td><b>'c': continue</b> - Run to the next breakpoint.<br>_</td><td><b>'l' : list</b> - Shows 11 lines of code<br>around the actual position.</td></tr><tr><td><b>'n': next</b> - Run the next line<br>in current function.</td><td><b>'s': step</b> - step in to a function.<br>_</td><td><b>'r' : return</b> - Continue execution until<br>the current function returns.</td></tr></tr><tr><td><b>'h': help</b> - Shows help for a command<br>or all (if none parameter)</td><td><b>'b': breakpoint</b> - Makes a breakpoint for<br>a function or codeline (nr).</td><td><b>'cl' : clear</b> - Deletes breakpoint(s)<br>_</td></tr></table>

# breakpoint
#from IPython.core.debugger import Pdb; Pdb().set_trace()
pass

#_sp: breakpoint - short

In [ ]:
#from IPython.core.debugger import Pdb; Pdb().set_trace()

#_sp: ANSI escape code

In [ ]:
#@markdown [ANSI escape code](https://en.wikipedia.org/wiki/ANSI_escape_code)
#@markdown * selection - [from here](https://gist.github.com/iansan5653/c4a0b9f5c30d74258c5f132084b78db9)

print('\x1b[0m Reset / Normal \x1b[0m')
print('\x1b[1m Bold or increased intensity \x1b[0m')
print('\x1b[3m Italic \x1b[0m')
print('\x1b[4m Underline \x1b[0m')

print(' Basic Foreground Colors:')
print('\t\x1b[30m Black foreground\x1b[0m')
print('\t\x1b[31m Red foreground\x1b[0m')
print('\t\x1b[32m Green foreground\x1b[0m')
print('\t\x1b[33m Yellow foreground\x1b[0m')
print('\t\x1b[34m Blue foreground\x1b[0m')
print('\t\x1b[35m Magenta foreground\x1b[0m')
print('\t\x1b[36m Cyan foreground\x1b[0m')
print('\t\x1b[37m White foreground\x1b[0m')
print('\t\x1b[39m Default foreground color \x1b[0m')

print(' Basic Background Colors:')
print('\t\x1b[40m Black background\x1b[0m')
print('\t\x1b[41m Red background\x1b[0m')
print('\t\x1b[42m Green background\x1b[0m')
print('\t\x1b[43m Yellow background\x1b[0m')
print('\t\x1b[44m Blue background\x1b[0m')
print('\t\x1b[45m Magenta background\x1b[0m')
print('\t\x1b[46m Cyan background\x1b[0m')
print('\t\x1b[47m White background\x1b[0m')
print('\t\x1b[49m Default background color \x1b[0m')

print(' Bright Foreground Colors:')
print('\t\x1b[90m Bright Black foreground\x1b[0m')
print('\t\x1b[91m Bright Red foreground\x1b[0m')
print('\t\x1b[92m Bright Green foreground\x1b[0m')
print('\t\x1b[93m Bright Yellow foreground\x1b[0m')
print('\t\x1b[94m Bright Blue foreground\x1b[0m')
print('\t\x1b[95m Bright Magenta foreground\x1b[0m')
print('\t\x1b[96m Bright Cyan foreground\x1b[0m')
print('\t\x1b[97m Bright White foreground\x1b[0m')

print(' Bright Background Colors:')
print('\t\x1b[100m Bright Black background\x1b[0m')
print('\t\x1b[101m Bright Red background\x1b[0m')
print('\t\x1b[102m Bright Green background\x1b[0m')
print('\t\x1b[103m Bright Yellow background\x1b[0m')
print('\t\x1b[104m Bright Blue background\x1b[0m')
print('\t\x1b[105m Bright Magenta background\x1b[0m')
print('\t\x1b[106m Bright Cyan background\x1b[0m')
print('\t\x1b[107m Bright White background\x1b[0m')

#_sp: record video

In [ ]:
#@markdown **record video** { form-width: "300px" }
# Adapted from: https://stackoverflow.com/a/62804023/4879610
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def record_video(filename):
  js = Javascript("""
    async function recordVideo() {
      // mashes together the advanced_outputs.ipynb function provided by Colab, 
      // a bunch of stuff from Stack overflow, and some sample code from:
      // https://developer.mozilla.org/en-US/docs/Web/API/MediaStream_Recording_API

      // Optional frames per second argument.
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      capture.textContent = "Start Recording";
      capture.style.background = "green";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({audio:true, video: true});
      // create a media recorder instance, which is an object
      // that will let you record what you stream.
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
      // Video is a media element.  This line here sets the object which serves
      // as the source of the media associated with the HTMLMediaElement
      // Here, we'll set it equal to the stream.
      video.srcObject = stream;

      // from https://stackoverflow.com/questions/62529304/is-there-any-way-to-capture-live-video-using-webcam-in-google-colab#comment114644726_62804023
      video.muted = true;

      // We're inside an async function, so this await will fire off the playing
      // of a video. It returns a Promise which is resolved when playback has 
      // been successfully started. Since this is async, the function will be 
      // paused until this has started playing. 
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      // and now, just wait for the capture button to get clicked in order to
      // start recording
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
      // use a promise to tell it to stop recording
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();

      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
      return btoa(binaryString);

    }
    """)
  try:
    display(js)
    data = eval_js('recordVideo({})')
    binary = b64decode(data)
    with open(filename, "wb") as video_file:
      video_file.write(binary)
    print(
        f"Finished recording video. Saved binary under filename in current working directory: {filename}"
    )
  except Exception as err:
      # In case any exceptions arise
      print(str(err))


from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 300):
  # show saved video in colab.
  video_file = open(video_path, "r+b").read()

  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")



video_path = "test.mp4"
record_video(video_path)



show_video(video_path)

#_sp: stream video

In [ ]:
#@markdown stream video
#https://www.quora.com/How-do-I-open-a-webcam-live-video-stream-in-Google-Colab

from IPython.display import HTML, Audio 
from google.colab.output import eval_js 
from base64 import b64decode 
import numpy as np 
import io 
from PIL import Image 
 
VIDEO_HTML = """ 
<video autoplay 
 width=%d height=%d style='cursor: pointer;'></video> 
<script> 
 
var video = document.querySelector('video') 
 
navigator.mediaDevices.getUserMedia({ video: true }) 
  .then(stream=> video.srcObject = stream) 
   
var data = new Promise(resolve=>{ 
  video.onclick = ()=>{ 
    var canvas = document.createElement('canvas') 
    var [w,h] = [video.offsetWidth, video.offsetHeight] 
    canvas.width = w 
    canvas.height = h 
    canvas.getContext('2d') 
          .drawImage(video, 0, 0, w, h) 
    video.srcObject.getVideoTracks()[0].stop() 
    video.replaceWith(canvas) 
    resolve(canvas.toDataURL('image/jpeg', %f)) 
  } 
}) 
</script> 
""" 
def take_photo(filename='photo.jpg', quality=0.8, size=(400,300)): 
  display(HTML(VIDEO_HTML % (size[0],size[1],quality))) 
  data = eval_js("data") 
  binary = b64decode(data.split(',')[1]) 
  f = io.BytesIO(binary) 
  return np.asarray(Image.open(f)) 
 
print ('click to take a photo...') 
img = take_photo() # click 
 
 
import matplotlib.pyplot as plt 
 
plt.figure(figsize=(5,5)) 
plt.imshow(img) 
plt.show()

#_sp: youtube

In [ ]:
#@markdown youtube jukebox { form-width: "200px" }
from IPython.display import YouTubeVideo

jukebox = {"The Dead South - In Hell I'll Be In Good Company"   :"B9FzVhw8_bY",
           "Big Bad Voodoo Daddy - Why Me?"                     :"a3Z4RWZa9WA",
           "Kishi Bashi - Violin Tsunami"                       :"xlXwpaAVoJQ",
           "Jamie T - Sheila"                                   :"4-L7Cadb-c0",
           "Tash Sultana - Jungle"                              :"1ExkpBpYEPw"}

select = "The Dead South - In Hell I'll Be In Good Company" #@param ["The Dead South - In Hell I'll Be In Good Company","Big Bad Voodoo Daddy - Why Me?","Kishi Bashi - Violin Tsunami","Jamie T - Sheila","Tash Sultana - Jungle"]

display ( YouTubeVideo ( jukebox [ select ] ) )

#_sp: radio buttons

In [ ]:
#@markdown radio buttons
import ipywidgets as widgets
import numpy


output_radio_selected = widgets.Text()
radio1 = widgets.RadioButtons(options=['mse'])
radio2 = widgets.RadioButtons(options=['mae'])

radio1.index = None
radio2.index = None

def radio1_observer(sender):
    #print(sender)
    radio2.unobserve(radio2_observer, names=['value'])
    radio2.index = None

    global selected_option
    output_radio_selected.value = radio1.value
    selected_option = output_radio_selected.value
    print('Selected option set to: ' + selected_option)

    radio2.observe(radio2_observer, names=['value'])

def radio2_observer(sender):
    radio1.unobserve(radio1_observer, names=['value'])
    radio1.index = None

    global selected_option2
    output_radio_selected.value = radio2.value
    selected_option2 = output_radio_selected.value
    print('Selected option set to: ' + selected_option2)

    radio1.observe(radio1_observer, names=['value'])


radio1.observe(radio1_observer, names=['value'])
radio2.observe(radio2_observer, names=['value'])

widgets.HBox([radio1,radio2])

#_sp: svg with graphviz

In [ ]:
#@markdown svg with graphviz
import graphviz
s='init->predict->loss->gradient->step->stop\nstep->predict[label=repeat]'
graphviz.Source('digraph G{ rankdir="LR"' + s + '; }')

# Text cell snippets

* only available via good old copy paste...

## tex and greek letters

$$\large \sum {\frac {Frit^z} {\sqrt[the] {C\alpha t}} }$$
<br>
<table>
<tr><td>Alpha $\alpha$ / $A$</td><td>Beta $\beta$ / $B$</td><td>Gamma $\gamma$ / $\Gamma$</td><td>Delta $\delta$ / $\Delta$</td><td>Epsilon $\epsilon$ / $E$</td><td>Zeta $\zeta$ / $Z$</td></tr>

<tr><td>Eta $\eta$ / $E$</td><td>Theta $\theta$ / $\Theta$</td><td>Iota $\iota$ / $I$</td><td>Kappa $\kappa$ / $K$</td><td>Lambda $\lambda$ / $\Lambda$</td><td>My $\mu$ / $M$</td></tr>

<tr><td>Ny $\nu$ / $N$</td><td>Xi $\xi$ / $\Xi$</td><td>Omikron $\omicron$ / $O$</td><td>Pi $\pi$ / $\Pi$</td><td>Rho $\rho$ / $R$</td><td>Sigma $\sigma$ / $\Sigma$</td></tr>

<tr><td>Tau $\tau$ / $T$</td><td>Ypsilon $\upsilon$ / $\Upsilon$</td><td>Phi $\phi$ / $\Phi$</td><td>Chi $\chi$ / $X$</td><td>Psi $\psi$ / $\Psi$</td><td>Omega $\omega$ / $\Omega$</td></tr>
</table>



## gdrive hosting 

1. Connect your google account. This allows you to save directly to your gdrive from within applications. 
2. Create new inkscape file (as an example) and save it to you gdrive. (It doesn't work with linux - you have to download, edit, upload) 
3. Open your gdrive in the browser, right click to the saved file and choose 'release'.
4. In the popup under 'general access' choose the option 'anyone who has the link'.
5. Copy the link. You will get the wrong format. It looks like <br>
```https://drive.google.com/file/d/1cehK_fxj8vtBX1df3aKWF6RSOygzIAiR/view?usp=sharing```
6. Change it to the following format: <br> ```https://drive.google.com/uc?id=1cehK_fxj8vtBX1df3aKWF6RSOygzIAiR```
7. After reload the notebook in the browser the changes are visible.

<br>You can use the link in a ```<img>``` tag<br>
<img src='https://drive.google.com/uc?id=1cehK_fxj8vtBX1df3aKWF6RSOygzIAiR' width="400">

or you can use it in your code.
```python
from IPython.display import SVG
SVG('https://drive.google.com/uc?id=1cehK_fxj8vtBX1df3aKWF6RSOygzIAiR')
```


## tables and lines

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td></tr>
<tr><td>r2d1</td><td>r2d2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td><td>r3d3</td></tr>
</table>

---
<table>
<tr><td colspan=3>r1d1 colspan3</td></tr>
<tr><td>r2d1</td><td>r2d2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td><td>r3d3</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td rowspan=3>r1d2<br>rowspan3</td><td>r1d3</td></tr>
<tr><td>r2d1</td><td>r2d2</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr>
<tr><td>r2d1</td><td colspan=2 rowspan=2>r2d2<br>colspan2<br>rowspan2</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
<tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr>
</table>

---
I - inner table / O - outer table
<table>
<tr><td>O r1d1</td><td>O r1d2</td><td>O r1d3</td><td>O r1d4</td></tr>
<tr><td>O r2d1</td><td colspan=2 rowspan=2 align=center>O r2d2  colspan2 rowspan2
<table>
<tr><td>I r1d1</td><td>I r1d2</td><td>I r1d3</td><td>I r1d4</td></tr>
<tr><td>I r2d1</td><td colspan=2 rowspan=2 align=center>
I r2d2<br>colspan2<br>rowspan2
</td><td>I r2d3</td></tr>
<tr><td>I r3d1</td><td>I r3d2</td></tr>
<tr><td>I r4d1</td><td>I r4d2</td><td>I r4d3</td><td>I r4d4</td></tr>
</table>
</td><td>O r2d3</td></tr>
<tr><td>O r3d1</td><td>O r3d2</td></tr>
<tr><td>O r4d1</td><td>O r4d2</td><td>O r4d3</td><td>O r4d4</td></tr>
</table>

---
<table>
<tr><td>r1d1</td><td>r1d2</td><td>r1d3</td><td>r1d4</td></tr>
<tr><td>r2d1</td><td colspan=2 rowspan=2 align=center>
$$\rule{7cm}{0.4pt}$$
$\left.{\;\\\;\\\;\\\;\\}\right\vert$
r2d2 - colspan2 - rowspan2$\left.{\;\\\;\\\;\\\;\\}\right\vert$<br>
$$\rule{7cm}{0.4pt}$$
</td><td>r2d3</td></tr>
<tr><td>r3d1</td><td>r3d2</td></tr>
<tr><td>r4d1</td><td>r4d2</td><td>r4d3</td><td>r4d4</td></tr>
</table>

---
<table><tr><td rowspan=4>
$\left.{\;\\\;\\\;\\}\right\vert$

</td><td colspan=2>$\rule{8cm}{0.4pt}$</td><td rowspan=4>
$\left.{\;\\\;\\\;\\}\right\vert$

</td></tr >
<tr><td  align=center>r2d1</td><td  align=center>r2d2</td></tr>
<tr><td  align=center>r3d1</td><td  align=center>r3d2</td></tr>
<tr><td colspan=2>
$\rule{8cm}{0.4pt}$

</td></tr></table>

---
<table><tr><td>

$\left.{\;}\right\vert$</td><td>

$\left.{\;\\}\right\vert$</td><td>

$\left.{\;\\\;\\}\right\vert$</td><td>

$\left.{\;\\\;\\\;\\}\right\vert$</td><td>

$\left.{\;\\\;\\\;\\\;\\\;\\}\right\vert$</td><td>

$\left.{\;\\\;\\\;\\\;\\\;\\\;\\}\right\vert$</td><td>

$\left.{\;\\\;\\\;\\\;\\\;\\\;\\\;\\}\right\vert$</td><td>

$\left.{\;\\\;\\\;\\\;\\\;\\\;\\\;\\\;\\}\right\vert$</td><td align=center>

$\rule{8cm}{0.4pt}$<br>
$\rule{10cm}{1pt}$<br>
$\rule{12cm}{4pt}$<br>
$\rule{10cm}{1pt}$<br>
$\rule{8cm}{0.4pt}$<br>

</td></tr></table>


# _